# Parsing csv to xlsx
## Data Sources
https://api.ehealth.gov.ua/api/services
## Commit git
- 3ed8a2e Start
- 2550448b First read json
- 75c0f119c modul json
- efbb4001 del dct
- 87d560a2 write parsing_1.xlsx parsing_2.xlsx
- 810eb9e6 write parsing_3.xlsx
- 780ca100 OK
- c98c8c1f write clas_6.xlsx


In [52]:
import pandas as pd
import re

df = pd.read_json('./data/processed/services.json', 
                  orient='records')
# конвертація datetime64[ns, UTC] в object ('2020-03-31') без часу
for i in ['inserted_at', 'updated_at']:
    df[i] = pd.to_datetime(df[i]).dt.date

df.to_excel("./data/interim/first.xlsx", index=False)

## Считування за допомогою модуля json

In [17]:
import json
import openpyxl

# считування json файла в пітоновський словник type(data) -> dict
with open('./data/raw/services.json', 'r', encoding='UTF-8') as file:
    dct = json.load(file)

with open('./data/processed/clas_6.json', 'r', encoding='UTF-8') as file:
    clas_6 = json.load(file)    

In [18]:
cl = ['Клас 5', 'Клас 14', 'Клас 17', 'Клас 4',
     'Клас 10', 'Клас 8', 'Клас 18', 'Клас 2',
     'Клас 9', 'Клас 15', 'Клас 19', 'Клас 12',
     'Клас 16', 'Клас 3', 'Клас 20', 'Клас 1',
     'Клас 13', 'Клас 11', 'Клас 7', 'LAB000',
     'Клас 6']

wb = openpyxl.Workbook()
ws = wb.active
ws['A1'] = 'cod'
ws['B1'] = 'description'
row = 2
for i in dct['data']:
    if i['code'] not in cl:
        ws.cell(row, column=1).value = i['code']
        ws.cell(row, column=2).value = i['name']
        row += 1
wb.save('./data/interim/parsing_1.xlsx')

In [19]:
wb = openpyxl.Workbook()
ws = wb.active
ws['A1'] = 'cod'
ws['B1'] = 'cod2'
ws['C1'] = 'description'
row = 2
for i in dct['data']:
    if i['code'] not in cl:
        for j in i['services']:
            ws.cell(row, column=1).value = i['name']
            ws.cell(row, column=2).value = i['code']
            ws.cell(row, column=3).value = j['code']
            ws.cell(row, column=4).value = j['name']
            row += 1
wb.save('./data/interim/parsing_2.xlsx')

In [20]:
wb1 = openpyxl.Workbook()
ws1 = wb1.active
ws1['A1'] = 'cod'
ws1['B1'] = 'cod2'
ws1['C1'] = 'cod3'
ws1['D1'] = 'description'

cll = cl[:-1]
row = 2
for i in dct['data']:
    if i['code'] in cll:
        for j in i['groups']:
            for k in j['groups']:
                for h in k['services']:
                    ws1.cell(row, column=1).value = i['code']
                    ws1.cell(row, column=2).value = j['code']
                    ws1.cell(row, column=3).value = k['code']
                    ws1.cell(row, column=4).value = k['name']
                    ws1.cell(row, column=5).value = h['category']
                    ws1.cell(row, column=6).value = h['code']
                    ws1.cell(row, column=7).value = h['name']
                    row += 1     
                    

wb1.save('./data/interim/parsing_3.xlsx')

In [21]:
wb = openpyxl.Workbook()
ws = wb.active

ws['A1'] = 'class'
ws['B1'] = 'class_name'
ws['C1'] = 'cod'
ws['D1'] = 'cod_name'
ws['E1'] = 'type'
ws['F1'] = 'cod_intervention'
ws['G1'] = 'intervention_name'

dct = clas_6['data'][0]
row = 2
for item in dct['groups']:
    for service in item['services']:
        ws.cell(row, column=1).value = dct['code']
        ws.cell(row, column=2).value = dct['name']
        ws.cell(row, column=3).value = item['code']
        ws.cell(row, column=4).value = item['name']
        ws.cell(row, column=5).value = service['category']
        ws.cell(row, column=6).value = service['code']
        ws.cell(row, column=7).value = service['name']
        row += 1 
        
                    
ws.column_dimensions['A'].width = 6 
ws.column_dimensions['B'].width = 26 
ws.column_dimensions['C'].width = 4 
ws.column_dimensions['D'].width = 64 
ws.column_dimensions['E'].width = 20 
ws.column_dimensions['F'].width = 16 
ws.column_dimensions['G'].width = 144 

wb.save('./data/processed/AKMI_clas_6.xlsx')    